In [1]:
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd
from tqdm import tqdm
from IPython.display import display
import numpy as np
from difflib import SequenceMatcher


In [2]:
url = 'https://www.ryuugames.com/category/visualnovel/english/'


In [3]:

# soup = BeautifulSoup(html, "lxml")

# main_page = soup.find(class_='td-ss-main-content')

# print(soup.find_all(class_ = ['entry-title td-module-title']))

# vision_url_list = []
# for item in soup.find_all(class_ = ['entry-title td-module-title']):
#     href = item.a.attrs['href']
#     vision_url_list.append(href)
# print(vision_url_list)

In [4]:
NUM_PAGES = 90
URL = 'https://www.ryuugames.com/category/visualnovel/english/'
vision_url_list = []
for page in range(1, NUM_PAGES):
    page_url = URL + 'page/' + str(page) + '/'
    #print(page_url)
    r = requests.get(page_url)

    r.encoding = 'utf-8'

    html = r.text
    
    soup = BeautifulSoup(html, "lxml")
    #print(soup.find_all(class_ = ['entry-title td-module-title']))

    for item in soup.find_all(class_ = ['entry-title td-module-title']):
        href = item.a.attrs['href']
        vision_url_list.append(href)



In [5]:
may_need_read = []

In [8]:
def extract_game_info(new_soup, url):
    vision_novel = False
    game_info = {}
    string_list = list(new_soup.strings)
    for i, text in enumerate(string_list):
        text = text.lstrip()
        
        if re.search("^https://vndb.org", text):
            vision_novel = True
            continue
        if text.startswith('Title'):
            saved_text = text
            title = ":".join(text.split(':')[1:])
            j = i+1
            if not title.strip():
                subtext = string_list[j]
                title = ":".join(subtext.split(':'))
                j += 1
            
            while title.endswith('\xa0'):
                
                subtitle = string_list[j]
                title = title + subtitle
                j+=1
            title = title.replace("\xa0", "")
            title = title.lstrip()
            game_info['Title'] = [title]
            continue
        if text.startswith('Original Title'):
            ori_title = ":".join(text.split(':')[1:])
            j = i+1
            if not ori_title.strip():
                subtext = string_list[i+1]
                ori_title = ":".join(subtext.split(':')[0:])
                j+=1
            while ori_title.endswith('\xa0'):

                subtitle = string_list[j]
                ori_title = ori_title + subtitle
                j+=1
            ori_title = ori_title.replace("\xa0", "")
            ori_title = ori_title.lstrip()
            game_info['Original Title'] = [ori_title]
            continue
        if text.startswith('Developer'):
            company = ":".join(text.split(':')[1:])
            if not company.strip():
                subtext = string_list[i+1]
                company = ":".join(subtext.split(':')[0:])
            company = company.lstrip()
            game_info['Developer'] = [company]
            continue
        if len(game_info.keys()) == 3 and vision_novel:
            break
    game_info['URL'] = [url]
    return game_info, vision_novel

In [9]:
all_res = {'Title':[], 'Original Title': [], 'Developer': [], 'URL' : []}
num_res = range(len(vision_url_list))
for i in tqdm(num_res):
    url = vision_url_list[i]
    r = requests.get(url)
        
    r.encoding = 'utf-8'

    html = r.text

    soup = BeautifulSoup(html, "lxml")

    #main_page = soup.find_all(dir='ltr')
    main_page = soup.find_all(class_ = ['td-post-content tagdiv-type'])
    new_soup = main_page[0]

    # div_index = -1
    # for j, ele in enumerate(new_soup):
    #     print(list(ele.strings))
    #     if 'INFO' in list(ele.strings):
    #         div_index = j
    #         break
    #print(i)
    
    game_info, vision_novel = extract_game_info(new_soup, url)
    if 'Title' not in game_info.keys():
        may_need_read.append(url)
        continue
    if 'Original Title' not in game_info.keys():
        game_info['Original Title'] = game_info['Title']
    if 'Developer' not in game_info.keys():
        game_info['Developer'] = ["NA"]
    if vision_novel:
        all_res = {key : list(all_res.get(key, []) + game_info.get(key, [])) for key in all_res.keys()}
    #print(vision_novel, game_info, all_res)
    

100%|██████████| 1334/1334 [22:35<00:00,  1.02s/it] 


In [10]:
df = pd.DataFrame(all_res)
df_no_ind = df.sort_values(by = ['Developer']).reset_index(drop=True)

display(df_no_ind)

,Title,Original Title,Developer,URL
0,Rose Guns Days – Last Season,ローズガンズデイズ ラストシーズン,07th Expansion,https://www.ryuugames.com/eng-rose-guns-days-l...
1,Rose Guns Days – Season 2,ローズガンズデイズ シーズン2,07th Expansion,https://www.ryuugames.com/eng-rose-guns-days-s...
2,Neko Kawaigari! ~Crane Inuneko Byouin Shinsats...,ネコっかわいがり! ～クレインイヌネコ病院診察中～,13cm,https://www.ryuugames.com/eng-neko-kawaigari-c...
3,"Chindere ~Kindly Offer Up Your Penis~, Chinder...",チンデレ ～お捧げなさい肉棒を～,1micron,https://www.ryuugames.com/eng-chindere-kindly-...
4,Total Nudity School Training / Zenra Toukou Tr...,全裸登校訓練,4H,https://www.ryuugames.com/eng-total-nudity-sch...
...,...,...,...,...
1009,Backstage Pass,Backstage Pass,sakevisual,https://www.ryuugames.com/backstage-pass-free-...
1010,Ookami Shoujo to Issho / Wolf Girl With You,狼少女といっしょ,seismic,https://www.ryuugames.com/ookami-shoujo-to-iss...
1011,"She Falls to a Perverted Bastard, Erogaki ni H...",エロガキに嵌まる彼女,shinaciku-castella,https://www.ryuugames.com/eng-she-falls-to-a-p...
1012,Koi to Senkyo to Chocolate High Resolution,恋と選挙とチョコレート High Resolution,sprite & Vridge Inc.,https://www.ryuugames.com/eng-koi-to-senkyo-to...


In [11]:
df_no_ind.to_excel('Game_list.xlsx')

In [ ]:
vision_url_list[129]

'https://www.ryuugames.com/grisaia-complete-edition-download/'

# Filter out old games

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
new_games = pd.read_excel('NewGame_list.xlsx', engine= 'openpyxl')

In [ ]:
df_no_ind = new_games.drop(columns = ["Unnamed: 0"]). sort_values(by = ['Developer']).reset_index(drop=True)

In [ ]:
old_games = pd.read_csv('old games.csv')

In [ ]:
old_game_list = np.array(old_games.iloc[:, 2].dropna())

In [ ]:
old_game_embedding = sbert_model.encode(old_game_list)


NameError: name 'old_game_list' is not defined

In [ ]:
def preprocess_string(tstring):
    tstring = tstring.lower()
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
 
    # Removing punctuations in string
    # Using loop + punctuation string
    for ele in tstring:
        if ele in punc:
            tstring = tstring.replace(ele, "")
    
    return tstring

def build_bigram(doc):
    bigrams = []
    for title in doc:
        title = preprocess_string(title).split(" ")

        for i in range(len(title)-1):
            bigram = title[i] + " " + title[i+1]
            bigrams.append(bigram)
    return bigrams


In [ ]:
old_game_bigram = build_bigram(old_game_list)

In [ ]:
def similar(a, b):
    a = preprocess_string(a)
    b = preprocess_string(b)
    score = SequenceMatcher(None, a, b).ratio()
    
    if a in b or b in a:
        score = 1.0
    # a_list = a.split(" ")
    # b_list = b.split(" ")
    # if len(list(set(a_list) & set(b_list))) != 0:
    #     return 1.0
    return score
def exist(a, doc, old_game_bigram):
    a_bigram = build_bigram([a])
    for biword in a_bigram:
        if biword in old_game_bigram:
            return True
    similarity_scores = [similar(a,x) for x in doc]
    return max(similarity_scores) >= 0.4

In [ ]:
remove_list = []
for index, row in tqdm(df_no_ind.iterrows()):
    title = row['Title']
    if exist(title, old_game_list, old_game_bigram):
        remove_list.append(index)

1014it [00:15, 65.26it/s]


In [ ]:
dropped_similar = df_no_ind.drop(df_no_ind.index[remove_list])
dropped_similar.to_excel('NewGame_list.xlsx')

In [ ]:
dropped_similar

,Title,Original Title,Developer
0,Rose Guns Days – Last Season,ローズガンズデイズ ラストシーズン,07th Expansion
1,Rose Guns Days – Season 2,ローズガンズデイズ シーズン2,07th Expansion
2,Neko Kawaigari! ~Crane Inuneko Byouin Shinsats...,ネコっかわいがり! ～クレインイヌネコ病院診察中～,13cm
3,"Chindere ~Kindly Offer Up Your Penis~, Chinder...",チンデレ ～お捧げなさい肉棒を～,1micron
4,Total Nudity School Training / Zenra Toukou Tr...,全裸登校訓練,4H
5,Steins;Gate Senkei Kousoku no Phenogram /STEIN...,STEINS;GATE 線形拘束のフェノグラム,5pb. Games
9,"Musumama 3, Itsu made mo Musuko no Mama ja Ira...",いつまでも息子のままじゃいられない！３ ～巨乳でムチムチな母さんのおっぱいを吸って揉んで挟ん...,ANIM Mother & Wife
10,"Itsumama 2, Itsu made mo Musuko no Mama ja Ira...",いつまでも息子のママじゃいられない！2～愛情たっぷり香夏子母さんのむっちりボディに射精管理さ...,ANIM Mother & Wife
11,"MUSUMAMA, Itsu made mo Musuko no Mama ja Irare...",いつまでも息子のままじゃいられない！ ～巨乳で美人な母さんは家では無防備すぎて僕の股間はもう...,ANIM Mother & Wife
12,"Rin to Shita Tsuma wa, Hitoshirezu Midarazuma ...",凛とした最愛妻は、人知れず淫乱ら妻へと堕ちて ～他の男を受け挿れ拡げられた濡れ穴は、もう俺の...,ANIM.teamMM


In [ ]:
similar('Kyonyuu Fantasy Gaiden / Funbag Fantasy : Sideboob Story', 'Funbag Fantasy')

0.4